In [166]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import networkx as nx

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [167]:
# === 1. Загрузка данных ===
# Загрузка нормализованной матрицы смежности
adj_matrix = pd.read_csv('data/los_adj.csv', header=None)
adj_matrix_torch = torch.tensor(adj_matrix.to_numpy(), dtype=torch.float32).to(device)

# Загрузка данных временных рядов
time_series_data = pd.read_csv('data/los_speed.csv')  # (2016, 207)
time_series_data_torch = torch.tensor(time_series_data.to_numpy(), dtype=torch.float32).to(device)

In [168]:
# === 2. Вычисление фич узлов ===
# Степень узлов
node_degree = adj_matrix_torch.sum(dim=1)

# Центральность узлов
node_centrality = node_degree / (adj_matrix_torch.shape[0] - 1)

# Коэффициент кластеризации
graph = nx.from_pandas_adjacency(adj_matrix)
clustering_coeffs = nx.clustering(graph)
node_clustering = torch.tensor([clustering_coeffs[i] for i in range(len(clustering_coeffs))], dtype=torch.float32).to(device)

# Сбор фич узлов
node_features = torch.stack([node_degree, node_centrality, node_clustering], dim=1).to(device)

In [169]:
# === 3. Подготовка данных для T-GCN ===
class TimeSeriesDataset(Dataset):
    def __init__(self, time_series, node_features, window_size, pred_size, mean=None, std=None):
        self.time_series = time_series
        self.node_features = node_features
        self.window_size = window_size
        self.pred_size = pred_size
        self.mean = mean
        self.std = std

    def normalize(self, data):
        # Нормализация: (data - mean) / std
        return (data - self.mean) / self.std

    def denormalize(self, data):
        # Де-нормализация: data * std + mean
        return data * self.std + self.mean

    def __len__(self):
        return len(self.time_series) - self.window_size - self.pred_size

    def __getitem__(self, idx):
        x = self.time_series[idx:idx + self.window_size]  # (window_size, nodes)
        y = self.time_series[idx + self.window_size:idx + self.window_size + self.pred_size]  # (pred_size, nodes)

        # Нормализация
        if self.mean is not None and self.std is not None:
            x = self.normalize(x)
            y = self.normalize(y)

        # Добавляем статические фичи узлов
        x = torch.stack([torch.cat([x_t.unsqueeze(-1), self.node_features], dim=-1) for x_t in x])  # (window_size, nodes, features + 3)
        return x, y

# Для нормализации используем среднее и стандартное отклонение по всем данным
mean = time_series_data_torch.mean(dim=0)  # Среднее по всем узлам
std = time_series_data_torch.std(dim=0)    # Стандартное отклонение по всем узлам

# Параметры
window_size = 12  # Количество временных шагов для входа
pred_size = 3     # Количество временных шагов для предсказания

dataset = TimeSeriesDataset(time_series_data_torch, node_features, window_size, pred_size, mean, std)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [170]:
# === 4. Определение модели T-GCN ===
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        # Отладочные сообщения для проверки размерностей

        support = torch.matmul(x, self.weight)  # Перемножение признаков и весов

        output = torch.matmul(adj, support)  # Применение матрицы смежности

        if self.bias is not None:
            output += self.bias

        return output


class TGCN(nn.Module):
    def __init__(self, in_features, hidden_gcn, hidden_gru, num_nodes, mean=None, std=None):
        super(TGCN, self).__init__()
        self.gcn = GraphConvolution(in_features, hidden_gcn)  # in_features = 4
        self.gru = nn.GRU(hidden_gcn * 207, hidden_gru, batch_first=True)  # Adjust input size to GRU
        self.fc = nn.Linear(hidden_gru, num_nodes)  # num_nodes = 207
        self.mean = mean
        self.std = std

    def normalize(self, data):
        # Нормализация: (data - mean) / std
        return (data - self.mean) / self.std

    def denormalize(self, data):
        # Де-нормализация: data * std + mean
        return data * self.std + self.mean

    def forward(self, x, adj):
        batch_size, time_steps, num_nodes, features = x.size()

        # Нормализуем данные перед подачей в модель
        if self.mean is not None and self.std is not None:
            x = self.normalize(x)

        # Reshape for GCN
        x = x.view(-1, num_nodes, features)  # [batch_size * time_steps, num_nodes, features]

        x = self.gcn(x, adj)  # GCN

        # Reshape for GRU
        x = x.view(batch_size, time_steps, -1)  # [batch_size, time_steps, num_nodes * hidden_gcn]

        out, _ = self.gru(x)  # GRU

        # Выбор последних pred_size временных шагов
        pred_size = 3  # Убедитесь, что это значение передаётся в модель
        out = out[:, -pred_size:, :]  # [batch_size, pred_size, hidden_gru]

        # Применение линейного слоя
        out = self.fc(out)

        # Де-нормализуем предсказания
        if self.mean is not None and self.std is not None:
            out = self.denormalize(out)

        return out


In [174]:
# === 5. Обучение модели ===
# Параметры модели
in_features = 4 # временные признаки + фичи узлов
hidden_gcn = 128
hidden_gru = 128
out_features = 207

model = TGCN(in_features=in_features, hidden_gcn=hidden_gcn, hidden_gru=hidden_gru, num_nodes=out_features).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005)
loss_fn = nn.MSELoss().to(device)

# Тренировка
epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for x_batch, y_batch in data_loader:
        optimizer.zero_grad()
        preds = model(x_batch, adj_matrix_torch)  # Прогнозы модели
        loss = loss_fn(preds, y_batch)      # Вычисление потерь
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    # if epoch % 10 == 0:
    print(f'Epoch {epoch + 1:>2}/{epochs}, Loss: {epoch_loss:.4f}')

Epoch  1/10, Loss: 65.2313
Epoch  2/10, Loss: 64.3854
Epoch  3/10, Loss: 64.8264
Epoch  4/10, Loss: 64.8362
Epoch  5/10, Loss: 64.5257
Epoch  6/10, Loss: 64.9085
Epoch  7/10, Loss: 64.2881
Epoch  8/10, Loss: 64.4411
Epoch  9/10, Loss: 64.2032
Epoch 10/10, Loss: 64.8294
